In [1]:
import json
from gen.utils.py_util import remove_spaces_and_lower
from vocab import Vocab
import revtok
import torch
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import models.nn.vnn as vnn
import torch. nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
torch.manual_seed(0)

In [2]:
with open('dataset.json', 'r') as fp:
    data_train = json.load(fp)
with open('dataset_valid_seen.json', 'r') as fp:
    data_valid_seen = json.load(fp)
with open('dataset_valid_unseen.json', 'r') as fp:
    data_valid_unseen = json.load(fp)
with open('dataset'+'_high_plan_train'+'.json', 'r') as fp:
    data_train_high_plan = json.load(fp)
with open('dataset'+'_high_plan_valid_seen'+'.json', 'r') as fp:
    data_valid_seen_high_plan = json.load(fp)
with open('dataset'+'_high_plan_valid_unseen'+'.json', 'r') as fp:
    data_valid_unseen_high_plan = json.load(fp)

with open('dataset'+'_high_plan_argument_train'+'.json', 'r') as fp:
    data_train_high_plan_argument = json.load(fp)
with open('dataset'+'_high_plan_argument_valid_seen'+'.json', 'r') as fp:
    data_valid_seen_high_plan_argument = json.load(fp)
with open('dataset'+'_high_plan_argument_valid_unseen'+'.json', 'r') as fp:
    data_valid_unseen_high_plan_argument = json.load(fp)

In [3]:
# data['look_at_obj_in_light']

In [3]:
class HighPlanClassificationDataset(Dataset):

    def __init__(self, dataset, labels = None):
      
        self.dataset = dataset
        if labels:
            self.label = labels
        else:
            self.label = list(dataset.keys())
        self.label.sort()
        print(self.label)
        self.data = []
        self.labels = []
        self.vocab = torch.load('pp.vocab')
        self.pad = 0
        self.seg = 1


        for key in dataset.keys():
         
            for item in dataset[key]:
                traj = {}
                traj['ann']  = {
                    'goal': revtok.tokenize(remove_spaces_and_lower(item))
                }
                traj['num'] = {}
                traj['num']['lang_goal'] = self.numericalize(self.vocab['word'], traj['ann']['goal'], train=True)
            
                lang_goal = traj['num']['lang_goal']
                
                # lang_goal_instr = lang_goal + lang_instr
                # lang_goal_instr = lang_instr

                lang_goal_instr = lang_goal
                traj['num']['lang_goal_instr'] = lang_goal_instr

                # for k, v in traj['num'].items():
                #             if k in {'lang_goal_instr'}:
                #                 # language embedding and padding
                #                 seqs = [torch.tensor(vv, device=device) for vv in v]
                #                 pad_seq = pad_sequence(seqs, batch_first=True, padding_value=self.pad)
                #                 seq_lengths = np.array(list(map(len, v)))
                #                 embed_seq = self.emb_word(pad_seq)
                #                 packed_input = pack_padded_sequence(embed_seq, seq_lengths, batch_first=True, enforce_sorted=False)
                #                 traj['num'][k] = packed_input

                self.data.append( lang_goal_instr  )
                self.labels.append(key)
                

    def serialize_lang_action(self, feat):
        '''
        append segmented instr language and low-level actions into single sequences
        '''
        is_serialized = not isinstance(feat['num']['lang_instr'][0], list)
        if not is_serialized:
            feat['num']['lang_instr'] = [word for desc in feat['num']['lang_instr'] for word in desc]

    @staticmethod
    def numericalize(vocab, words, train=True):
        '''
        converts words to unique integers
        '''
        return vocab.word2index([w.strip().lower() for w in words], train=train)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
     
        
        return self.data[idx], self.label.index(self.labels[idx])

train_plan = HighPlanClassificationDataset(data_train_high_plan)
valid_seen_plan = HighPlanClassificationDataset(data_valid_seen_high_plan)
valid_unseen_plan = HighPlanClassificationDataset(data_valid_unseen_high_plan)

train_plan_argument = HighPlanClassificationDataset(data_train_high_plan_argument)
valid_seen_plan_argument = HighPlanClassificationDataset(data_valid_seen_high_plan_argument, train_plan_argument.label)
valid_unseen_plan_argument = HighPlanClassificationDataset(data_valid_unseen_high_plan_argument, train_plan_argument.label)


['CleanObject', 'CoolObject', 'GotoLocation', 'HeatObject', 'PickupObject', 'PutObject', 'SliceObject', 'ToggleObject']
['CleanObject', 'CoolObject', 'GotoLocation', 'HeatObject', 'PickupObject', 'PutObject', 'SliceObject', 'ToggleObject']
['CleanObject', 'CoolObject', 'GotoLocation', 'HeatObject', 'PickupObject', 'PutObject', 'SliceObject', 'ToggleObject']
['alarmclock', 'apple', 'armchair', 'baseballbat', 'basketball', 'bathtubbasin', 'bed', 'book', 'bottle', 'bowl', 'box', 'bread', 'butterknife', 'cabinet', 'candle', 'cart', 'cd', 'cellphone', 'cloth', 'coffeemachine', 'coffeetable', 'countertop', 'creditcard', 'cup', 'desk', 'desklamp', 'diningtable', 'dishsponge', 'drawer', 'dresser', 'egg', 'floorlamp', 'fork', 'fridge', 'garbagecan', 'handtowel', 'handtowelholder', 'kettle', 'keychain', 'knife', 'ladle', 'laptop', 'lettuce', 'microwave', 'mug', 'newspaper', 'ottoman', 'pan', 'pen', 'pencil', 'peppershaker', 'pillow', 'plate', 'plunger', 'pot', 'potato', 'remotecontrol', 'safe', 

In [4]:
len(train_plan_argument.label)

77

In [5]:

class TaskClassificationDataset(Dataset):

    def __init__(self, dataset):
      
        self.dataset = dataset
        self.label = list(dataset.keys())
        self.label.sort()
        print(self.label)
        self.data = []
        self.labels = []
        self.vocab = torch.load('pp.vocab')
        self.pad = 0
        self.seg = 1


        for key in dataset.keys():
            for item in dataset[key]:
                for p in item:
                    traj = {}
                    traj['ann']  = {
                        'goal': revtok.tokenize(remove_spaces_and_lower(p['task_desc'])) + ['<<goal>>'],
                        'instr': [revtok.tokenize(remove_spaces_and_lower(x)) for x in p['high_descs']] + [['<<stop>>']],
                    }
                    traj['num'] = {}
                    traj['num']['lang_goal'] = self.numericalize(self.vocab['word'], traj['ann']['goal'], train=True)
                    traj['num']['lang_instr'] = [self.numericalize(self.vocab['word'], x, train=True) for x in traj['ann']['instr']]
                    self.serialize_lang_action(traj)
                    lang_goal, lang_instr = traj['num']['lang_goal'], traj['num']['lang_instr']
                    lang_goal_instr = lang_goal + lang_instr
                    # lang_goal_instr = lang_instr
                    # lang_goal_instr = lang_goal
                    traj['num']['lang_goal_instr'] = lang_goal_instr
                    # for k, v in traj['num'].items():
                    #             if k in {'lang_goal_instr'}:
                    #                 # language embedding and padding
                    #                 seqs = [torch.tensor(vv, device=device) for vv in v]
                    #                 pad_seq = pad_sequence(seqs, batch_first=True, padding_value=self.pad)
                    #                 seq_lengths = np.array(list(map(len, v)))
                    #                 embed_seq = self.emb_word(pad_seq)
                    #                 packed_input = pack_padded_sequence(embed_seq, seq_lengths, batch_first=True, enforce_sorted=False)
                    #                 traj['num'][k] = packed_input

                    self.data.append( lang_goal_instr  )

                    self.labels.append(key)

    def serialize_lang_action(self, feat):
        '''
        append segmented instr language and low-level actions into single sequences
        '''
        is_serialized = not isinstance(feat['num']['lang_instr'][0], list)
        if not is_serialized:
            feat['num']['lang_instr'] = [word for desc in feat['num']['lang_instr'] for word in desc]

    @staticmethod
    def numericalize(vocab, words, train=True):
        '''
        converts words to unique integers
        '''
        return vocab.word2index([w.strip().lower() for w in words], train=train)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
     
        
        return self.data[idx], self.label.index(self.labels[idx])

In [6]:

train_set = TaskClassificationDataset(data_train )
val_set1 = TaskClassificationDataset(data_valid_seen )
val_set2 = TaskClassificationDataset(data_valid_unseen)


class Net(nn.Module):
    def __init__(self, num_output=7):
        super().__init__()
        self.lang_dropout = nn.Dropout(0.3, inplace=True)
        self.enc = nn.LSTM(512, 512, bidirectional=True, batch_first=True)
        self.enc_att = vnn.SelfAttn(512 * 2)
        self.fc1 = nn.Linear(1024,num_output)
       
        self.emb_word = nn.Embedding(len(train_set.vocab['word']), 512)
    def forward(self, x):
        emb_lang_goal_instr = x
        self.lang_dropout(emb_lang_goal_instr.data)
        enc_lang_goal_instr, _ = self.enc(emb_lang_goal_instr)
        enc_lang_goal_instr, _ = pad_packed_sequence(enc_lang_goal_instr, batch_first=True)
        self.lang_dropout(enc_lang_goal_instr)
        cont_lang_goal_instr = self.enc_att(enc_lang_goal_instr)
        res = self.fc1(cont_lang_goal_instr)
        res = nn.functional.relu(res)

        # res = self.fc2(res)
        # res = nn.functional.relu(res)

        res = F.log_softmax(res, dim=1)
        return res
model = Net().cuda()


def collate_fn(batch):
        device = 'cuda'
        lang_goal_instr, gt = zip(*batch)
        seqs = [torch.tensor(vv, device=device) for vv in lang_goal_instr]
        pad_seq = pad_sequence(seqs, batch_first=True, padding_value=0)
        seq_lengths = np.array(list(map(len, lang_goal_instr)))
        embed_seq = model.emb_word(pad_seq)
        packed_input = pack_padded_sequence(embed_seq, seq_lengths, batch_first=True, enforce_sorted=False)
        lang_goal_instr = packed_input
        return lang_goal_instr, gt 

    

['look_at_obj_in_light', 'pick_and_place_simple', 'pick_and_place_with_movable_recep', 'pick_clean_then_place_in_recep', 'pick_cool_then_place_in_recep', 'pick_heat_then_place_in_recep', 'pick_two_obj_and_place']
['look_at_obj_in_light', 'pick_and_place_simple', 'pick_and_place_with_movable_recep', 'pick_clean_then_place_in_recep', 'pick_cool_then_place_in_recep', 'pick_heat_then_place_in_recep', 'pick_two_obj_and_place']
['look_at_obj_in_light', 'pick_and_place_simple', 'pick_and_place_with_movable_recep', 'pick_clean_then_place_in_recep', 'pick_cool_then_place_in_recep', 'pick_heat_then_place_in_recep', 'pick_two_obj_and_place']


21025


In [7]:
from torch.utils.data import DataLoader


train_loader = DataLoader( train_set, batch_size=100, collate_fn=collate_fn, shuffle=True, num_workers=0)
val_loader1 = DataLoader( val_set1, batch_size=100, collate_fn=collate_fn, shuffle=False, num_workers=0)
val_loader2 = DataLoader( val_set2, batch_size=100, collate_fn=collate_fn, shuffle=False, num_workers=0)

train_loader_plan = DataLoader( train_plan, batch_size=100, collate_fn=collate_fn, shuffle=True, num_workers=0)
valid_seen_loader_plan = DataLoader( valid_seen_plan, batch_size=100, collate_fn=collate_fn, shuffle=True, num_workers=0)
valid_unseen_loader_plan = DataLoader( valid_unseen_plan, batch_size=100, collate_fn=collate_fn, shuffle=True, num_workers=0)

train_loader_plan_argument = DataLoader( train_plan_argument, batch_size=200, collate_fn=collate_fn, shuffle=True, num_workers=0)
valid_seen_loader_plan_argument = DataLoader( valid_seen_plan_argument, batch_size=200, collate_fn=collate_fn, shuffle=True, num_workers=0)
valid_unseen_loader_plan_argument = DataLoader( valid_unseen_plan_argument, batch_size=200, collate_fn=collate_fn, shuffle=True, num_workers=0)

In [9]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [10]:
model = Net(num_output=7).cuda()

In [8]:

import torch.optim as optim
import torch.nn.functional as F
from transformers import get_linear_schedule_with_warmup

criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
epochs = 20
optimizer = torch.optim.AdamW(model.parameters(), 3e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, 50, len(train_loader)*epochs)
for epoch  in range(epochs):
    running_loss = 0.0
    correct = 0
    model.train()
    for i, sample_batched in enumerate(train_loader):
        optimizer.zero_grad()

        lang_goal_instr, gt = sample_batched
        target = torch.LongTensor(gt).cuda()
        output = model(lang_goal_instr.cuda())
        loss =  F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        scheduler.step()
        running_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
    print('[%d, %5d] loss: %.3f correct: %.3f' % (epoch + 1, i + 1, running_loss, correct/len(train_loader.dataset) ))
    # print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / len(train_loader.dataset)))
            
    
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i, sample_batched in enumerate(val_loader1):
           

            lang_goal_instr, gt = sample_batched
            target = torch.LongTensor(gt).cuda()
            output = model(lang_goal_instr.cuda())

            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(val_loader1.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(val_loader1.dataset),
    100. * correct / len(val_loader1.dataset)))

    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i, sample_batched in enumerate(val_loader2):
          

            lang_goal_instr, gt = sample_batched
            target = torch.LongTensor(gt).cuda()
            output = model(lang_goal_instr.cuda())

            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(val_loader2.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(val_loader2.dataset),
    100. * correct / len(val_loader2.dataset)))

KeyboardInterrupt: 

In [12]:
model_plan = Net(8).cuda()
import torch.optim as optim
import torch.nn.functional as F
from transformers import get_linear_schedule_with_warmup

criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
epochs = 20
optimizer_plan = torch.optim.AdamW(model_plan.parameters(), 3e-5)
scheduler_plan = get_linear_schedule_with_warmup(optimizer_plan, 50, len(train_loader_plan)*epochs)
for epoch  in range(epochs):
    running_loss = 0.0
    correct = 0
    model_plan.train()
    for i, sample_batched in enumerate(train_loader_plan):
        optimizer_plan.zero_grad()

        lang_goal_instr, gt = sample_batched
        target = torch.LongTensor(gt).cuda()
        output = model_plan(lang_goal_instr.cuda())
        loss =  F.nll_loss(output, target)
        loss.backward()
        optimizer_plan.step()
        scheduler_plan.step()
        running_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
    print('[%d, %5d] loss: %.3f correct: %.3f' % (epoch + 1, i + 1, running_loss, correct/len(train_loader_plan.dataset) ))
    # print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / len(train_loader.dataset)))
            
    
    model_plan.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i, sample_batched in enumerate(valid_seen_loader_plan):
           

            lang_goal_instr, gt = sample_batched
            target = torch.LongTensor(gt).cuda()
            output = model_plan(lang_goal_instr.cuda())

            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(valid_seen_loader_plan.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(valid_seen_loader_plan.dataset),
    100. * correct / len(valid_seen_loader_plan.dataset)))

    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i, sample_batched in enumerate(valid_unseen_loader_plan):
          

            lang_goal_instr, gt = sample_batched
            target = torch.LongTensor(gt).cuda()
            output = model_plan(lang_goal_instr.cuda())

            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(valid_unseen_loader_plan.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(valid_unseen_loader_plan.dataset),
    100. * correct / len(valid_unseen_loader_plan.dataset)))

[1,  1413] loss: 591.551 correct: 0.881

Test set: Average loss: 0.1374, Accuracy: 5368/5570 (96%)


Test set: Average loss: 0.1565, Accuracy: 4936/5140 (96%)

[2,  1413] loss: 159.562 correct: 0.976

Test set: Average loss: 0.1142, Accuracy: 5476/5570 (98%)


Test set: Average loss: 0.1238, Accuracy: 5039/5140 (98%)

[3,  1413] loss: 107.833 correct: 0.986

Test set: Average loss: 0.0706, Accuracy: 5492/5570 (99%)


Test set: Average loss: 0.0783, Accuracy: 5063/5140 (99%)

[4,  1413] loss: 76.420 correct: 0.990

Test set: Average loss: 0.0677, Accuracy: 5498/5570 (99%)


Test set: Average loss: 0.0744, Accuracy: 5068/5140 (99%)

[5,  1413] loss: 71.349 correct: 0.991

Test set: Average loss: 0.0646, Accuracy: 5500/5570 (99%)


Test set: Average loss: 0.0709, Accuracy: 5069/5140 (99%)

[6,  1413] loss: 67.647 correct: 0.991

Test set: Average loss: 0.0623, Accuracy: 5505/5570 (99%)


Test set: Average loss: 0.0683, Accuracy: 5069/5140 (99%)

[7,  1413] loss: 65.607 correct: 0.991

Tes

In [8]:
num_class = 77
model_plan_argument = Net(num_class).cuda()
import torch.optim as optim
import torch.nn.functional as F
from transformers import get_linear_schedule_with_warmup

criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
epochs = 50
optimizer_plan_argument = torch.optim.AdamW(model_plan_argument.parameters(), 3e-5)
scheduler_plan_argument = get_linear_schedule_with_warmup(optimizer_plan_argument, 50, len(train_loader_plan_argument)*epochs)

for epoch  in range(epochs):
    confusion_matrix = torch.zeros(num_class, num_class)
    running_loss = 0.0
    correct = 0
    model_plan_argument.train()
    for i, sample_batched in enumerate(train_loader_plan_argument):
        optimizer_plan_argument.zero_grad()

        lang_goal_instr, gt = sample_batched
        target = torch.LongTensor(gt).cuda()
        output = model_plan_argument(lang_goal_instr.cuda())
        loss =  F.nll_loss(output, target)
        loss.backward()
        optimizer_plan_argument.step()
        scheduler_plan_argument.step()
        running_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        
        for t, p in zip(torch.tensor(gt).view(-1), pred.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
    print(confusion_matrix.diagonal()/confusion_matrix.sum(1))
    # print(confusion_matrix[79])
    print('[%d, %5d] loss: %.3f correct: %.3f' % (epoch + 1, i + 1, running_loss, correct/len(train_loader_plan_argument.dataset) ))
    # print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / len(train_loader.dataset)))
            
    
    model_plan_argument.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i, sample_batched in enumerate(valid_seen_loader_plan_argument):
           

            lang_goal_instr, gt = sample_batched
            target = torch.LongTensor(gt).cuda()
            output = model_plan_argument(lang_goal_instr.cuda())

            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(valid_seen_loader_plan_argument.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(valid_seen_loader_plan_argument.dataset),
    100. * correct / len(valid_seen_loader_plan_argument.dataset)))

    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i, sample_batched in enumerate(valid_unseen_loader_plan_argument):
          

            lang_goal_instr, gt = sample_batched
            target = torch.LongTensor(gt).cuda()
            output = model_plan_argument(lang_goal_instr.cuda())

            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(valid_unseen_loader_plan_argument.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(valid_unseen_loader_plan_argument.dataset),
    100. * correct / len(valid_unseen_loader_plan_argument.dataset)))

tensor([0.0000, 0.5047, 0.0613, 0.0000, 0.0000, 0.0405, 0.0000, 0.0979, 0.3134,
        0.1428, 0.1088, 0.2581, 0.3765, 0.3909, 0.0056, 0.0000, 0.0000, 0.0000,
        0.0131, 0.0041, 0.0052, 0.7646, 0.3383, 0.1135, 0.0000, 0.0000, 0.6592,
        0.0065, 0.3070, 0.0137, 0.0861, 0.4369, 0.0000, 0.0000, 0.3689, 0.0331,
           nan, 0.0000, 0.2374, 0.2229, 0.0000, 0.0000, 0.2193, 0.2765, 0.0000,
        0.0000, 0.0000, 0.2155, 0.1890, 0.0018, 0.0000, 0.0000, 0.1124, 0.0000,
        0.0848, 0.6442, 0.2649, 0.0000, 0.0000, 0.1511, 0.0000, 0.6041, 0.1562,
        0.0000, 0.0026, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2825, 0.0938,
        0.0000, 0.6502, 0.0000, 0.0000, 0.0101])


IndexError: index 79 is out of bounds for dimension 0 with size 77

tensor([2, 2])